# Comparación de Funciones de Activación en una CNN (MNIST)

---

## **Introducción**

En el aprendizaje profundo, las **funciones de activación** son un componente esencial de las redes neuronales, ya que permiten introducir **no linealidad** en los modelos y, con ello, la capacidad de **aprender patrones complejos**.

Este notebook compara el comportamiento de tres funciones de activación clásicas:
- **ReLU (Rectified Linear Unit)**  
- **Tanh (Tangente hiperbólica)**  
- **Sigmoid**

Todas son evaluadas dentro de la **misma red neuronal convolucional (CNN)** entrenada sobre el conjunto de datos **MNIST**, que contiene imágenes en escala de grises de dígitos escritos a mano (0–9).

---

## **Conceptos generales**

- **Función de activación:**  
  Determina cómo responde cada neurona a la información que recibe. Su papel es introducir *no linealidad*, permitiendo que la red aprenda relaciones complejas.  

- **CNN (Convolutional Neural Network):**  
  Tipo de red diseñada para procesar datos con estructura espacial (como imágenes). Utiliza *capas convolucionales* que detectan bordes, formas y patrones.

- **MNIST:**  
  Dataset de imágenes de **dígitos escritos a mano** (28×28 px) usado tradicionalmente para evaluar modelos de visión por computadora.

- **Precisión (Accuracy):**  
  Porcentaje de aciertos del modelo al clasificar imágenes nuevas (de prueba).

- **Saturación de gradientes:**  
  Problema común en redes profundas donde algunas funciones (como **Sigmoid**) reducen el valor del gradiente casi a cero, dificultando el aprendizaje.

---


### 1. Importación de librerías y configuración del entorno

In [1]:
import torch                      # Núcleo de PyTorch (tensores, GPU, etc.)
import torch.nn as nn             # Capas y módulos para redes neuronales
import torch.optim as optim       # Algoritmos de optimización (SGD, Adam, etc.)
import torchvision                # Datasets y utilidades para visión por computadora
import torchvision.transforms as transforms  # Transformaciones para preprocesar imágenes
import time                       # Medir tiempo de entrenamiento
import numpy as np                # Operaciones numéricas adicionales

print(f"Usando PyTorch v{torch.__version__}")

# Configurar Dispositivo (SOLO CUDA)
# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("¡Éxito! Usando dispositivo: cuda (GPU)")
# else:
#     print("ERROR: CUDA no está disponible.")
#     print("Este script requiere una GPU NVIDIA.")
#     exit() # Salir si no hay CUDA

# Selección automática del dispositivo (GPU si está disponible, de lo contrario CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando:", device)

Usando PyTorch v2.9.0+cpu
Usando: cpu


### 2. Carga y preparación del dataset (MNIST)

In [ ]:

# 📚 2. Carga y preparación del dataset (MNIST)

# Transformaciones que se aplican a cada imagen:
# 1) ToTensor() → convierte la imagen (0–255) en un tensor (0–1)
# 2) Normalize() → ajusta los valores a un rango [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Cargar el dataset MNIST (dígitos escritos a mano)
# root: carpeta donde se almacenan los datos
# train=True  → conjunto de entrenamiento (60,000 imágenes)
# train=False → conjunto de prueba (10,000 imágenes)
# download=True → lo descarga si no existe
# transform → aplica las transformaciones definidas arriba
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)

# DataLoaders → cargan los datos en mini-lotes (batches)
# batch_size: cantidad de imágenes por iteración
# shuffle=True → mezcla los datos en cada época
# num_workers: hilos de carga (2 = más rápido en CPU)
BATCH_SIZE = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)


100.0%
100.0%
100.0%
100.0%


### 3. Definición del modelo CNN profundo

In [ ]:

# 🧩 3. Definición del modelo CNN profundo

def build_deep_cnn_model(activation_module):
    """
    Crea una red neuronal convolucional (CNN) profunda.
    
    Parámetros:
      activation_module : módulo de activación de PyTorch (nn.ReLU(), nn.Tanh(), nn.Sigmoid(), etc.)
                          Define qué función de activación se aplicará entre capas.
    """
    model = nn.Sequential(

        # ---------- Bloque 1 ----------
        nn.Conv2d(
            in_channels=1,   # número de canales de entrada (1 para imágenes en escala de grises)
            out_channels=16, # número de filtros (mapas de características) que generará esta capa
            kernel_size=3,   # tamaño del filtro convolucional (3x3)
            padding=1        # agrega un “borde” de 1 píxel para conservar el tamaño original
        ),
        activation_module,

        nn.Conv2d(
            in_channels=16,  # entrada: 16 canales del paso anterior
            out_channels=16, # salida: mantiene 16 canales
            kernel_size=3,
            padding=1
        ),
        activation_module,

        nn.MaxPool2d(
            kernel_size=2,   # reduce el tamaño de la imagen a la mitad (2x2)
            stride=2         # desplazamiento de la ventana del maxpool
        ),  # salida: 16 x 14 x 14


        # ---------- Bloque 2 ----------
        nn.Conv2d(
            in_channels=16, out_channels=32,
            kernel_size=3, padding=1
        ),
        activation_module,

        nn.Conv2d(
            in_channels=32, out_channels=32,
            kernel_size=3, padding=1
        ),
        activation_module,

        nn.MaxPool2d(
            kernel_size=2, stride=2
        ),  # salida: 32 x 7 x 7


        # ---------- Bloque 3 ----------
        nn.Conv2d(
            in_channels=32, out_channels=64,
            kernel_size=3, padding=1
        ),
        activation_module,

        nn.Conv2d(
            in_channels=64, out_channels=64,
            kernel_size=3, padding=1
        ),
        activation_module,

        nn.Conv2d(
            in_channels=64, out_channels=64,
            kernel_size=3, padding=1
        ),
        activation_module,


        # ---------- Clasificador ----------
        nn.Flatten(),  # convierte el tensor 3D (64x7x7) en un vector 1D (3136 valores)

        nn.Linear(
            in_features=64 * 7 * 7,  # número de entradas (neuronas aplanadas)
            out_features=128         # número de salidas (neuronas en la capa oculta)
        ),
        activation_module,

        nn.Linear(
            in_features=128,  # número de entradas (de la capa anterior)
            out_features=10   # número de clases de salida (dígitos 0–9)
        )
    )

    # Mueve el modelo al dispositivo correcto (CPU o GPU)
    return model.to(device)


### ⚙️ 4. Funciones de entrenamiento y evaluación

In [ ]:

# Función de Entrenamiento
def train_model(model, trainloader, criterion, optimizer, epochs=3):
    """
    Entrena el modelo con los datos de entrenamiento.
    
    Parámetros:
      model      : red neuronal a entrenar
      trainloader: DataLoader con los lotes (batches) de entrenamiento
      criterion  : función de pérdida (por ej. CrossEntropyLoss)
      optimizer  : algoritmo que ajusta los pesos (Adam, SGD, etc.)
      epochs     : número de épocas de entrenamiento
    """
    model.train()  # activa el modo entrenamiento (permite dropout, batchnorm, etc.)

    for epoch in range(epochs):
        running_loss = 0.0  # acumulador de pérdida por época

        # i = índice del batch | data = (inputs, labels)
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            # Mover los tensores al dispositivo (CPU o GPU)
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()        # reinicia los gradientes acumulados
            outputs = model(inputs)      # inferencia (forward pass)
            loss = criterion(outputs, labels)  # calcula la pérdida
            loss.backward()              # retropropagación (calcula gradientes)
            optimizer.step()             # actualiza los pesos del modelo

            running_loss += loss.item()  # acumula pérdida promedio

        print(f"  [Época {epoch + 1}/{epochs}] Pérdida: {running_loss / len(trainloader):.3f}")

# Función de Evaluación
def evaluate_model(model, testloader):
    """
    Evalúa el modelo en el conjunto de prueba.
    
    Parámetros:
      model      : red neuronal ya entrenada
      testloader : DataLoader con los datos de prueba
    
    Retorna:
      accuracy (%) del modelo sobre el conjunto de prueba
    """
    model.eval()  # modo evaluación (desactiva dropout y batchnorm)
    correct = 0
    total = 0

    # torch.no_grad() → desactiva el cálculo de gradientes (más rápido y eficiente)
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)                # predicciones
            _, predicted = torch.max(outputs.data, 1)  # índice de la clase con mayor probabilidad
            total += labels.size(0)                # total de imágenes evaluadas
            correct += (predicted == labels).sum().item()  # conteo de aciertos

    accuracy = 100 * correct / total
    return accuracy



### 5. Entrenamiento y comparación de funciones de activación

In [5]:
# Diccionario con las funciones de activación a probar.
# La clave es el nombre (string) y el valor es el módulo de PyTorch.
activations_to_test = {
    'relu': nn.ReLU(),      # Rectified Linear Unit
    'tanh': nn.Tanh(),      # Tangente hiperbólica
    'sigmoid': nn.Sigmoid() # Función sigmoide
}

training_results = {}  # guardará los resultados finales de cada activación
EPOCHS = 3             # número de épocas para todas las pruebas (igual para comparar)

print(f"\nIniciando comparación (Red PROFUNDA, {EPOCHS} épocas)...\n")

# Iterar sobre cada activación del diccionario
for name, activation_module in activations_to_test.items():
    print(f"--- ENTRENANDO CON: {name.upper()} ---")

    # Construir el modelo con la activación correspondiente
    model = build_deep_cnn_model(activation_module)

    # Función de pérdida para clasificación multiclase (usa logits sin softmax)
    criterion = nn.CrossEntropyLoss()

    # Optimizador Adam: ajusta los pesos del modelo en cada paso
    optimizer = optim.Adam(model.parameters())

    # Medir tiempo total de entrenamiento
    start_time = time.time()
    train_model(model, trainloader, criterion, optimizer, epochs=EPOCHS)
    total_time = time.time() - start_time

    # Evaluar el modelo en el conjunto de prueba
    final_accuracy = evaluate_model(model, testloader)

    # Guardar resultados para esta activación
    training_results[name] = {
        "time_seconds": total_time,
        "final_accuracy": final_accuracy
    }

    # Mostrar resultados intermedios
    print(f"Tiempo: {total_time:.2f} segundos")
    print(f"Precisión: {final_accuracy:.2f} %\n")



Iniciando comparación (Red PROFUNDA, 3 épocas)...

--- ENTRENANDO CON: RELU ---
  [Época 1/3] Pérdida: 0.259
  [Época 2/3] Pérdida: 0.057
  [Época 3/3] Pérdida: 0.041
Tiempo: 88.45 segundos
Precisión: 99.06 %

--- ENTRENANDO CON: TANH ---
  [Época 1/3] Pérdida: 0.198
  [Época 2/3] Pérdida: 0.076
  [Época 3/3] Pérdida: 0.063
Tiempo: 94.50 segundos
Precisión: 98.69 %

--- ENTRENANDO CON: SIGMOID ---
  [Época 1/3] Pérdida: 2.308
  [Época 2/3] Pérdida: 2.303
  [Época 3/3] Pérdida: 2.302
Tiempo: 88.80 segundos
Precisión: 11.35 %



### 6. Resultados finales de la comparación

In [6]:
# Encabezado general
print("--- RESULTADOS (Red PROFUNDA) ---")
print(f"Entrenamiento sobre {EPOCHS} épocas (Batch Size: {BATCH_SIZE})\n")

# Formato de tabla con columnas alineadas
print(f"{'Función':<10} | {'Tiempo (seg)':<15} | {'Precisión Final (%)':<20}")
print("-" * 50)

# Recorrer el diccionario con los resultados guardados
for activation, results in training_results.items():
    # Imprimir nombre de la activación, tiempo y precisión formateados
    print(f"{activation:<10} | {results['time_seconds']:<15.2f} | {results['final_accuracy']:<20.2f}")


--- RESULTADOS (Red PROFUNDA) ---
Entrenamiento sobre 3 épocas (Batch Size: 64)

Función    | Tiempo (seg)    | Precisión Final (%) 
--------------------------------------------------
relu       | 88.45           | 99.06               
tanh       | 94.50           | 98.69               
sigmoid    | 88.80           | 11.35               
